In [17]:
from sibyl.core import Sibyl
from mongoengine import disconnect

In [18]:
database = "housing"

In [19]:
disconnect()
config = {
    "mongodb": {
        "db": database,
        "host": "localhost",
        "port": 27017,
        "username": None,
        "password": None,
    },
    "log_filename": "test.csv",
    "feature_distribution_location": None,
    "flask": {},
}
explorer = Sibyl(config, docker=False)
app = explorer._init_flask_app("test")
client = app.test_client()

In [20]:
# TEST ENTITY APIs
eid = None

response = client.get("/api/v1/entities/")
assert response.status_code == 200
assert 'entities' in response.json
print("Number of entities:",   len(response.json['entities']))

if eid is None:
    eid = response.json["entities"][0]["eid"]
eids = [response.json["entities"][i]["eid"] for i in range(0, 3)]

response = client.get("/api/v1/entities/" + eid + "/")
assert response.status_code == 200
assert response.json["eid"] == eid
assert "features" in response.json
assert "property" in response.json
sample_entity = response.json

Number of entities: 2441


In [21]:
# TEST FEATURE APIs

response = client.get("/api/v1/features/")
assert response.status_code == 200
assert "features" in response.json
print("Number of features:", len(response.json["features"]))
sample_feature = response.json['features'][0]

response = client.get("/api/v1/features/" + sample_feature["name"] + "/")
assert response.status_code == 200
assert "name" in response.json
print("Sample feature:", response.json)

response = client.get("/api/v1/categories/")
assert response.status_code == 200
assert "categories" in response.json
print("Sample category", response.json["categories"][0])

Number of features: 1459
Sample feature: {'name': 'COUNT(scada)', 'description': 'The number of all instances of "scada" for each "COD_ELEMENT" in "turbines".', 'type': 'numeric', 'negated_description': None, 'category': 'all'}
Sample category {'name': 'all', 'color': None, 'abbreviation': None}


In [22]:
# TEST MODEL APIs
response = client.get("/api/v1/models/")
assert response.status_code == 200
print("Number of models:", len(response.json["models"]))
sample_model = response.json["models"][0]
model_id = sample_model["id"]

response = client.get("/api/v1/models/" + model_id + "/")
assert response.status_code == 200
print("Sample model:", response.json)

response = client.get("/api/v1/importance/?model_id=" + model_id)
assert response.status_code == 200
assert "importances" in response.json
print("Sample importance", next(iter(response.json["importances"].items())))

response = client.get("/api/v1/prediction/?model_id=" + model_id + "&eid=" + eid)
assert response.status_code == 200
print("Sample prediction:", response.json)
prediction = response.json["output"]

Number of models: 1
Sample model: {'id': '64b6fa42f3b206d0c5598839', 'name': 'placeholder', 'description': 'placeholder', 'performance': 'placeholder'}
Sample importance ('COUNT(scada)', 0.0)
Sample prediction: {'output': 1}


In [23]:
# TEST COMPUTING
import pandas as pd

response = client.post("/api/v1/contributions/", json={"eid": eid, "model_id": model_id})
assert response.status_code == 200
assert "contributions" in response.json
print("Sample contribution", next(iter(response.json["contributions"].items())))

response = client.post("/api/v1/multi_contributions/", json={"eids": eids, "model_id": model_id})
assert response.status_code == 200
assert "contributions" in response.json
for eid in response.json["contributions"]:
    try:
        pd.read_json(response.json["contributions"][eid], orient="index")
    except Exception:
        print("Error with multi-contributions - wrong format")

features_options = list(sample_entity["features"].keys())
changes = [(features_options[0], sample_entity["features"][features_options[0]]),
           (features_options[1], sample_entity["features"][features_options[1]])]
response = client.post("/api/v1/modified_prediction/",json={"eid": eid, "model_id": model_id, "changes": changes})
assert response.status_code == 200
assert "prediction" in response.json
print("Sample modified prediction:", response.json)

response = client.post("/api/v1/single_change_predictions/", json={"eid": eid, "model_id": model_id, "changes": changes})
assert response.status_code == 200
assert "changes" in response.json
print("Sample changes:", response.json["changes"])

Sample contribution ('COUNT(scada)', 0.0)
Sample modified prediction: {'prediction': 0}
Sample changes: [['COUNT(scada)', 0], ['MAX(scada.WCNV_Torq_max)', 0]]


In [24]:
# TEST COMPUTING - PART 2

response = client.post("/api/v1/similar_entities/", json={"eids": eids, "model_id": model_id})
assert response.status_code == 200
assert "similar_entities" in response.json
print_flag = True
for eid in response.json["similar_entities"]:
    assert "X" in response.json["similar_entities"][eid]
    try:
        X = pd.read_json(response.json["similar_entities"][eid]["X"], orient="index").head()
        if print_flag:
            print("Sample similar entity:\n", X)
    except Exception:
        print("Error with similar entities - wrong X format")
    assert "y" in response.json["similar_entities"][eid]
    try:
        y = pd.read_json(response.json["similar_entities"][eid]["y"], orient="index").head()
        if print_flag:
            print(y)
            print_flag=False
    except Exception:
        print("Error with similar entities - wrong y format")

Sample similar entity:
      COUNT(scada)  MAX(scada.WCNV_Torq_max)  MAX(scada.WCNV_Torq_mean)  \
384             0                402.832765                 400.635769   
349             0                402.832765                 400.635769   
385             0                402.832765                 400.635769   

     MAX(scada.WCNV_Torq_sd)  MAX(scada.WGEN_ClSt_max)  \
384                62.474973                  4.057511   
349                62.474973                  4.057511   
385                62.474973                  4.057511   

     MAX(scada.WGEN_ClSt_mean)  MAX(scada.WGEN_ClSt_min)  \
384                   3.548899                  3.175252   
349                   3.548899                  3.175252   
385                   3.548899                  3.175252   

     MAX(scada.WGEN_IntTmp_max)  MAX(scada.WGEN_IntTmp_mean)  \
384                   57.647153                    54.940089   
349                   57.647153                    54.940089   
385          